In [ ]:
!pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 46.2 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import GRB

###Base Code

In [ ]:
model = gp.Model()

Q_A = model.addVar(name="Q_A")  # Quantity grown in environment A
Q_B = model.addVar(name="Q_B")  # Quantity grown in environment B
Q_Agent1_A = model.addVar(name="Q_Agent1_A", lb=0) # Quantity of A that baked by Agent 1
Q_Agent1_B = model.addVar(name="Q_Agent1_B", lb=0) # Quantity of B that baked by Agent 1
Q_Agent2_A = model.addVar(name="Q_Agent2_A", lb=0) # Quantity of A that baked by Agent 2
Q_Agent2_B = model.addVar(name="Q_Agent2_B", lb=0) # Quantity of B that baked by Agent 2
T_H = model.addVar(name="T_H")  # Quantity grown in environment A
T_B = model.addVar(name="T_B")  # Quantity grown in environment B

# harvesting constraint
model.addConstr(Q_A + Q_B >= 1000 )
# overtime harvesting contraint
model.addConstr(Q_A + Q_B <= 1000 + T_H)
# baking_constraint
model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B >= 800)
# overtime_baking_constraint
model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B <= 800 + T_B)

model.addConstr(0.44 * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
model.addConstr(0.72 * Q_B == Q_Agent1_B + Q_Agent2_B, "harvested_B_constraint")

# Overtime constraints
model.addConstr(T_H <= 400 )
model.addConstr(T_B <= 300)

# Good critters ratio constraint
model.addConstr(
    ((Q_Agent1_A * 0.50 + Q_Agent2_A * 0.40) + (Q_Agent1_B * 0.60 + Q_Agent2_B * 0.30)) >= 0.4 * (Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B),
    "good_critters_ratio_constraint"
)

# Bad critters ratio constraint
model.addConstr(
    ((Q_Agent1_A * 0.05 + Q_Agent2_A * 0.06) + (Q_Agent1_B * 0.10 + Q_Agent2_B * 0.06)) <= 0.07 * (Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B),
    "bad_critters_ratio_constraint"
)

# Final Product Objective (Minimize Cost)
total_cost = 11 * Q_A + 17.5 * Q_B + 4 * T_H + 18 * (Q_Agent1_A + Q_Agent1_B) + 12.5 * (Q_Agent2_A + Q_Agent2_B) + 6.5 * T_B
total_revenue = 100 * (Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B)

model.setObjective(
    total_revenue - total_cost,
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Display the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Solution Found:")
    print(f"Q_A: {Q_A.x}")
    print(f"Q_B: {Q_B.x}")
    print(f"Q_Agent1_A: {Q_Agent1_A.x}")
    print(f"Q_Agent1_B: {Q_Agent1_B.x}")
    print(f"Q_Agent2_A: {Q_Agent2_A.x}")
    print(f"Q_Agent2_B: {Q_Agent2_B.x}")
    print(f"Overtime for harvesting: {T_H.x}")
    print(f"Overtime for baking: {T_B.x}")
    print(f"Total Profit: ${model.objVal}")
else:
    print("No optimal solution found.")

Restricted license - for non-production use only - expires 2024-10-28
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0x9ec60317
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.02s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5874981e+05   5.198840e+02   0.000000e+00      0s
       5    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.04 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Optimal Solution Found:
Q_A: 196.87499999999994
Q_B: 1203.125
Q_Agent1_A: 86.624999

###Q4

In [ ]:
import pandas as pd
percent_increase_values = []
profits = []
# # Loop over different percentage increases
for i in range(1, 11):
    model = gp.Model()
    Q_A = model.addVar(name="Q_A")  # Quantity grown in environment A
    Q_B = model.addVar(name="Q_B")  # Quantity grown in environment B
    Q_Agent1_A = model.addVar(name="Q_Agent1_A", lb=0) # Quantity of A that baked by Agent 1
    Q_Agent1_B = model.addVar(name="Q_Agent1_B", lb=0) # Quantity of B that baked by Agent 1
    Q_Agent2_A = model.addVar(name="Q_Agent2_A", lb=0) # Quantity of A that baked by Agent 2
    Q_Agent2_B = model.addVar(name="Q_Agent2_B", lb=0) # Quantity of B that baked by Agent 2
    T_H = model.addVar(name="T_H")  # Quantity grown in environment A
    T_B = model.addVar(name="T_B")  # Quantity grown in environment B
    improved_perc = i * 0.01
    # Update the constraints that depend on the percentage increase
    model.addConstr(Q_A + Q_B >= 1000 )
    model.addConstr(Q_A + Q_B <= 1000 + T_H)
    model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B >= 800)
    model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B <= 800 + T_B)
    # model.addConstr(0.44 * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
    model.addConstr((0.44 + improved_perc) * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
    model.addConstr(0.72 * Q_B == Q_Agent1_B + Q_Agent2_B, "harvested_B_constraint")
    model.addConstr(T_H <= 400 )
    model.addConstr(T_B <= 300)
    model.addConstr(
        ((Q_Agent1_A * 0.50 + Q_Agent2_A * 0.40) + (Q_Agent1_B * 0.60 + Q_Agent2_B * 0.30)) >= 0.4 * (Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B),
        "good_critters_ratio_constraint"
    )
    model.addConstr(
        ((Q_Agent1_A * 0.05 + Q_Agent2_A * 0.06) + (Q_Agent1_B * 0.10 + Q_Agent2_B * 0.06)) <= 0.07 * (Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B),
        "bad_critters_ratio_constraint"
    )

    total_cost = 11 * Q_A + 17.5 * Q_B + 4 * T_H + 18 * (Q_Agent1_A + Q_Agent1_B) + 12.5 * (Q_Agent2_A + Q_Agent2_B) + 6.5 * T_B
    total_revenue = 100 * (Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B)

    model.setObjective(
        total_revenue - total_cost,
        GRB.MAXIMIZE
    )
    model.optimize()
    # Display the results
    if model.status == GRB.OPTIMAL:
        profits.append(model.objVal)
        percent_increase_values.append(improved_perc * 100)  # Store the percentage increase
    else:
        print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0x2b5bd111
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6158258e+05   5.263557e+02   0.000000e+00      0s
       6    5.5868276e+04   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.586827586e+04
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 ph

In [ ]:
# Create a DataFrame to store the results
result_df = pd.DataFrame({
    'Percentage Increase': percent_increase_values,
    'Profit': profits
})

# Display the DataFrame
display(result_df)

,Percentage Increase,Profit
0,1.0,55868.275862
1,2.0,56071.789474
2,3.0,56267.793358
3,4.0,56456.695652
4,5.0,56638.875445
5,6.0,56814.685315
6,7.0,56984.453608
7,8.0,57148.486486
8,9.0,57307.069767
9,10.0,57460.470588


In [ ]:
percent_increase_values = []
profits = []
# # Loop over different percentage increases
for i in range(1, 6):
    model = gp.Model()
    Q_A = model.addVar(name="Q_A")  # Quantity grown in environment A
    Q_B = model.addVar(name="Q_B")  # Quantity grown in environment B
    Q_Agent1_A = model.addVar(name="Q_Agent1_A", lb=0) # Quantity of A that baked by Agent 1
    Q_Agent1_B = model.addVar(name="Q_Agent1_B", lb=0) # Quantity of B that baked by Agent 1
    Q_Agent2_A = model.addVar(name="Q_Agent2_A", lb=0) # Quantity of A that baked by Agent 2
    Q_Agent2_B = model.addVar(name="Q_Agent2_B", lb=0) # Quantity of B that baked by Agent 2
    T_H = model.addVar(name="T_H")  # Quantity grown in environment A
    T_B = model.addVar(name="T_B")  # Quantity grown in environment B
    improved_perc = i * 0.01
    # Update the constraints that depend on the percentage increase
    model.addConstr(Q_A + Q_B >= 1000 )
    model.addConstr(Q_A + Q_B <= 1000 + T_H)
    model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B >= 800)
    model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B <= 800 + T_B)
    model.addConstr(0.44 * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
    # model.addConstr((0.44 + improved_perc) * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
    model.addConstr((0.72 + improved_perc) * Q_B == Q_Agent1_B + Q_Agent2_B, "harvested_B_constraint")
    model.addConstr(T_H <= 400 )
    model.addConstr(T_B <= 300)
    model.addConstr(
        ((Q_Agent1_A * 0.50 + Q_Agent2_A * 0.40) + (Q_Agent1_B * 0.60 + Q_Agent2_B * 0.30)) >= 0.4 * (Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B),
        "good_critters_ratio_constraint"
    )
    model.addConstr(
        ((Q_Agent1_A * 0.05 + Q_Agent2_A * 0.06) + (Q_Agent1_B * 0.10 + Q_Agent2_B * 0.06)) <= 0.07 * (Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B),
        "bad_critters_ratio_constraint"
    )

    total_cost = 11 * Q_A + 17.5 * Q_B + 4 * T_H + 18 * (Q_Agent1_A + Q_Agent1_B) + 12.5 * (Q_Agent2_A + Q_Agent2_B) + 6.5 * T_B
    total_revenue = 100 * (Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B)

    model.setObjective(
        total_revenue - total_cost,
        GRB.MAXIMIZE
    )
    model.optimize()
    # Display the results
    if model.status == GRB.OPTIMAL:
        profits.append(model.objVal)
        percent_increase_values.append(improved_perc * 100)  # Store the percentage increase
    else:
        print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0x19fc637b
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6048986e+05   5.214864e+02   0.000000e+00      0s
       5    5.6568795e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.656879532e+04
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 ph

In [ ]:
# Create a DataFrame to store the results
result_df = pd.DataFrame({
    'Percentage Increase': percent_increase_values,
    'Profit': profits
})

# Display the DataFrame
display(result_df)

,Percentage Increase,Profit
0,1.0,56568.795322
1,2.0,57477.229572
2,3.0,58382.135922
3,4.0,59283.534884
4,5.0,60181.446809


###Q5

In [ ]:
import pandas as pd
from gurobipy import Model, GRB

def create_model():
    # Initialize a new model
    model = Model("CrispyCrittersInc")

    # Add your variables
    Q_A = model.addVar(name="Q_A")  # Quantity grown in environment A
    Q_B = model.addVar(name="Q_B")  # Quantity grown in environment B
    # M = 1000000  # A large constant for binary constraint
    #over_AB = model.addVar(vtype=GRB.BINARY, name="over_AB")  # Binary variable for overtime production
    Q_Agent1_A = model.addVar(name="Q_Agent1_A", lb=0) # Quantity of A that baked by Agent 1
    Q_Agent1_B = model.addVar(name="Q_Agent1_B", lb=0) # Quantity of B that baked by Agent 1
    Q_Agent2_A = model.addVar(name="Q_Agent2_A", lb=0) # Quantity of A that baked by Agent 2
    Q_Agent2_B = model.addVar(name="Q_Agent2_B", lb=0) # Quantity of B that baked by Agent 2
    #over_12 = model.addVar(vtype=GRB.BINARY, name="over_12") # Binary variable for overtime baking
    T_H = model.addVar(name="T_H")  # Quantity grown in environment A
    T_B = model.addVar(name="T_B")  # Quantity grown in environment B

    M = 1000000 # Set a really large number

    # harvesting constraint
    model.addConstr(Q_A + Q_B >= 1000 )
    # overtime harvesting contraint
    model.addConstr(Q_A + Q_B <= 1000 + T_H)
    # baking_constraint
    model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B >= 800)
    # overtime_baking_constraint
    model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B <= 800 + T_B)
    # logistic: ma <= A <= Ma, a is a binary variable
    model.addConstr(0.44 * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
    model.addConstr(0.72 * Q_B == Q_Agent1_B + Q_Agent2_B, "harvested_B_constraint")
    model.addConstr(T_H <= 400 )
    model.addConstr(T_B <= 300)

    # Define good_critters, bad_critters, and total_critters within the function
    good_critters = (Q_Agent1_A * 0.50 + Q_Agent2_A * 0.40) + (Q_Agent1_B * 0.60 + Q_Agent2_B * 0.30)
    bad_critters = (Q_Agent1_A * 0.05 + Q_Agent2_A * 0.06) + (Q_Agent1_B * 0.10 + Q_Agent2_B * 0.06)
    total_critters = Q_Agent1_A + Q_Agent2_A + Q_Agent1_B + Q_Agent2_B

    total_cost = 11 * Q_A + 17.5 * Q_B + 4 * T_H + 18 * (Q_Agent1_A + Q_Agent1_B) + 12.5 * (Q_Agent2_A + Q_Agent2_B) + 6.5 * T_B
    total_revenue = 100 * (Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B)

    return model, total_cost, total_revenue, good_critters, bad_critters, total_critters


# Define the range for the changes
bad_critters_range = range(7, 11)  # From 7% to 10%
good_critters_range = range(40, 34, -1)  # From 40% to 35%

# Initialize a DataFrame to store the results
results = pd.DataFrame(columns=['Good_Critters%', 'Bad_Critters%', 'Profit'])

for bad_percent in bad_critters_range:
    for good_percent in good_critters_range:
        model, total_cost, total_revenue, good_critters, bad_critters, total_critters = create_model()

        # Set the objective function using the variables
        model.setObjective(total_revenue - total_cost, GRB.MAXIMIZE)

        # Add modified critter ratio constraints
        model.addConstr(good_critters >= good_percent * 0.01 * total_critters, "GoodCrittersRatio")
        model.addConstr(bad_critters <= bad_percent * 0.01 * total_critters, "BadCrittersRatio")

        # Optimize the model
        model.optimize()

        # Debugging: Print model status and key variable values if the model is optimal
        if model.status == GRB.OPTIMAL:
            print(f"Optimal solution found for Good_Critters%: {good_percent}, Bad_Critters%: {bad_percent}")
            print(f"Total Cost: {total_cost.getValue()}, Total Revenue: {total_revenue.getValue()}")
            results = results.append({
                'Good_Critters%': good_percent,
                'Bad_Critters%': bad_percent,
                'Profit': model.objVal
            }, ignore_index=True)
        else:
            print(f"Model not optimal or infeasible for Good_Critters%: {good_percent}, Bad_Critters%: {bad_percent}")
            print("Model Status:", model.status)

print(results)


Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 29 nonzeros
Model fingerprint: 0x9ec60317
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 25 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.5874981e+05   5.198840e+02   0.000000e+00      0s
       5    5.5656812e+04   0.000000e+00   0.000000e+00      0s

Solved in 5 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.565681250e+04
Optimal solution found for Good_Critters%: 40, Bad_Critters%: 7
Total Cost: 39630.6875, Total Revenue: 95287.5
Gurobi Optimizer version 10.0.3 build v10.

<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-

Coefficient statistics:
  Matrix range     [2e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6130814e+05   5.317570e+02   0.000000e+00      0s
       4    5.9269600e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.02 seconds (0.00 work units)
Optimal objective  5.926960000e+04
Optimal solution found for Good_Critters%: 38, Bad_Critters%: 8
Total Cost: 41530.4, Total Revenue: 100800.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 30 nonzeros
Model fingerprint: 0x5dca6f63
Coefficient statistics:
  Matrix range     [

<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-

Solved in 4 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.963920000e+04
Optimal solution found for Good_Critters%: 36, Bad_Critters%: 9
Total Cost: 41160.8, Total Revenue: 100800.0
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 8 columns and 30 nonzeros
Model fingerprint: 0x83c064d5
Coefficient statistics:
  Matrix range     [1e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 4 rows and 2 columns
Presolve time: 0.01s
Presolved: 6 rows, 6 columns, 26 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.6130814e+05   5.317570e+02   0.000000e+00      0s
       4    5.9824000e+04   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.01 seconds (0.0

<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-9-19aed4724e49>:73: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
<ipython-input-

###Q6

In [ ]:
# Create a new model
model = gp.Model()

# Define decision variables
Q_A = model.addVar(name="Q_A")  # Quantity grown in environment A
Q_B = model.addVar(name="Q_B")  # Quantity grown in environment B
Q_Agent1_A = model.addVar(name="Q_Agent1_A", lb=0)  # Quantity of A that baked by Agent 1
Q_Agent1_B = model.addVar(name="Q_Agent1_B", lb=0)  # Quantity of B that baked by Agent 1
Q_Agent2_A = model.addVar(name="Q_Agent2_A", lb=0)  # Quantity of A that baked by Agent 2
Q_Agent2_B = model.addVar(name="Q_Agent2_B", lb=0)  # Quantity of B that baked by Agent 2
T_H = model.addVar(name="T_H", lb=0)  # Overtime for harvesting
T_B = model.addVar(name="T_B", lb=0)  # Overtime for baking
Staff_H = model.addVar(name="Staff_H", lb=0, vtype=GRB.INTEGER)  # Staffing for harvesting
Staff_B = model.addVar(name="Staff_B", lb=0, vtype=GRB.INTEGER)  # Staffing for baking


# Constraints
model.addConstr(Q_A + Q_B >= 1000, "harvesting_capacity_constraint")
model.addConstr(Q_A + Q_B <= 1000 + T_H + Staff_H, "overtime_harvesting_constraint") ######
model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B >= 800, "baking_capacity_constraint")
model.addConstr(Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B <= 800 + T_B + Staff_B , "overtime_baking_constraint") ######

model.addConstr(0.44 * Q_A == Q_Agent1_A + Q_Agent2_A, "harvested_A_constraint")
model.addConstr(0.72 * Q_B == Q_Agent1_B + Q_Agent2_B, "harvested_B_constraint")

model.addConstr(T_H <= 400, "harvesting_overtime_limit")
model.addConstr(T_B <= 300, "baking_overtime_limit")

#model.addConstr(Staff_H <= 1000, "harvesting_staff_limit") ######
#model.addConstr(Staff_B <= 800, "baking_staff_limit") ######

model.addConstr(
    ((Q_Agent1_A * 0.50 + Q_Agent2_A * 0.40) + (Q_Agent1_B * 0.60 + Q_Agent2_B * 0.30)) >= 0.4 * (Q_A + Q_B),
    "good_critters_ratio_constraint"
)

model.addConstr(
    ((Q_Agent1_A * 0.05 + Q_Agent2_A * 0.06) + (Q_Agent1_B * 0.10 + Q_Agent2_B * 0.06)) <= 0.07 * (Q_A + Q_B),
    "bad_critters_ratio_constraint"
)

# Update the objective function with the new cost structure
total_cost = 11 * Q_A + 17.5 * Q_B + 4 * T_H + 18 * (Q_Agent1_A + Q_Agent1_B) + 12.5 * (Q_Agent2_A + Q_Agent2_B) + 6.5 * T_B + 30 * Staff_H + 20 * Staff_B ######
total_revenue = 100 * (Q_Agent1_A + Q_Agent1_B + Q_Agent2_A + Q_Agent2_B)

model.setObjective(
    total_revenue - total_cost,
    GRB.MAXIMIZE
)

# Optimize the model
model.optimize()

# Display the results
if model.status == GRB.OPTIMAL:
    print(f"Optimal Solution Found:")
    print(f"Q_A: {Q_A.x}")
    print(f"Q_B: {Q_B.x}")
    print(f"Q_Agent1_A: {Q_Agent1_A.x}")
    print(f"Q_Agent1_B: {Q_Agent1_B.x}")
    print(f"Q_Agent2_A: {Q_Agent2_A.x}")
    print(f"Q_Agent2_B: {Q_Agent2_B.x}")
    print(f"Overtime for harvesting: {T_H.x}")
    print(f"Overtime for baking: {T_B.x}")
    print(f"Staffing for harvesting: {Staff_H.x}")
    print(f"Staffing for baking: {Staff_B.x}")
    print(f"Total Revenue: ${model.objVal}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (linux64)

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 10 rows, 10 columns and 36 nonzeros
Model fingerprint: 0x3d7b04d1
Variable types: 8 continuous, 2 integer (0 binary)
Coefficient statistics:
  Matrix range     [5e-02, 1e+00]
  Objective range  [4e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+02, 1e+03]
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 8 rows, 10 columns, 30 nonzeros
Variable types: 8 continuous, 2 integer (0 binary)
Found heuristic solution: objective 56025.333333

Root relaxation: objective 5.697685e+04, 8 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 56976.8519    0    1 56025.3333 569